In [1]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
# from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer, IBMQ
# from qiskit.circuit import Parameter
# from qiskit.visualization import plot_histogram
# Getting rid of unnecessary warnings
import warnings
from matplotlib.cbook import MatplotlibDeprecationWarning
warnings.filterwarnings('ignore', category=MatplotlibDeprecationWarning)
# from qiskit.tools.visualization import plot_histogram
# Importing standard Qiskit libraries
# from qiskit.compiler import transpile, assemble
#from qiskit.tools.jupyter import *
# from qiskit.visualization import *
# from qiskit.circuit.library.standard_gates import XGate
# from qiskit.circuit.library.standard_gates import RZGate
import math
from scipy import linalg
from numpy.linalg import solve
import scipy.stats
from qulacs import QuantumState
from qulacs import QuantumCircuit
from qulacs.gate import DenseMatrix
from qulacs.circuit import QuantumCircuitOptimizer
# from qulacsvis import circuit_drawer

#並列処理をするために
from multiprocessing import Pool
from multiprocessing import Process

from qulacs.gate import Identity, X,Y,Z #パウリ演算子
from qulacs.gate import H,S,Sdag, sqrtX,sqrtXdag,sqrtY,sqrtYdag #1量子ビット Clifford演算
from qulacs.gate import T,Tdag #1量子ビット 非Clifford演算
from qulacs.gate import RX,RY,RZ #パウリ演算子についての回転演算
from qulacs.gate import CNOT, CZ, SWAP #2量子ビット演算
"""
def function(hoge):
    #やりたいこと
    return x

def multi(n):
    p = Pool(10) #最大プロセス数:10
    result = p.map(function, range(n))
    return result

def main():
    data = multi(20)
    for i in data:
        print(i)

main()

この場合だと、「0,1,2・・・19と値を変化させてfunctionを20回実行する」という処理になります。
functionの返値はresultにリストで入っているので、受け取って標準出力しています。

また、私の環境では12コア（正確には6コア12スレッド）まで使えるので最大プロセス数は10としました。
最大まで使うとブラウザを開くのも一苦労になるのでやめておいた方が無難です。


"""

def bin4(m,a,b):#mをa桁の2進数にした時のb桁目
    m=int(m)
    #print(f'(m,a,b)=({m},{a},{b})')
    l = [*map(int,f"{format(m,'b'):>0{a}}")]
    return l[a-1-b]

def bin5(m,a):#mをa桁の2進数にした時の配列を返す
    m=int(m)
    #print(f'(m,a,b)=({m},{a},{b})')
    l = [*map(int,f"{format(m,'b'):>0{a}}")]
    return l
def bin6(m,a):#mをa桁の2進数にした時の配列(文字列)を返す
    m=int(m)
    #print(f'(m,a,b)=({m},{a},{b})')
    l = [*map(int,f"{format(m,'b'):>0{a}}")]
    l=str(l).replace('[','').replace(']','').replace(',','').replace(' ','')
    return l

def decimal(n,a):#長さnのaという01配列を10進数に変換。
    answer=0
    for i in range(n):
        answer+=pow(2,n-1-i)*a[i]
        #answer+=pow2(n-1-i)*a[i]
    return answer


<ipython-input-1-54ea843062d2>:9: MatplotlibDeprecationWarning: MatplotlibDeprecationWarning was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use matplotlib.MatplotlibDeprecationWarning instead.
  from matplotlib.cbook import MatplotlibDeprecationWarning


In [2]:
import sympy
# 円周率
pi = sympy.pi
# 自然対数の底
E = sympy.E
# 虚数単位
I = sympy.I
# オイラーの等式
# E**(I*pi)==-1
from qulacs.gate import DenseMatrix

def MCRZ(bit,n,state,degree):
    m=sum(bit[0:n])
    a=[0 for i in range(m+1)]
    count=0
    for i in range(n):
        if bit[i]==1:
            a[count]=i
            count+=1
    a[m]=bit[n]#a[m]にtargetbitの場所
    mcrz=np.array([[0+0j for i in range(2**(m+1))] for j in range(2**(m+1))])
    for i in range(2**(m+1)-1):
        mcrz[i][i]=1
    # mcrz[2*m][2*m]=E**(-I*degree/2)
    mcrz[2**(m+1)-1][2**(m+1)-1]=E**(I*degree)

    # gate_array=[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]]
    print(len(mcrz))
    print(a)
    # mcrz_gate = DenseMatrix(a,mcrz[0:4][0:4])
    mcrz_gate = DenseMatrix(a,mcrz)
    mcrz_gate.update_quantum_state(state)



# print(state.get_vector())
# MCRZ([1,0,1],2,state,pi)
# print(state.get_vector())

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()
zeros_original=np.zeros((100,28,28))
ones_original=np.zeros((100,28,28))
i=0
j=0
k=0
while i<100 or j<100:
    if train_labels[k]==0 and i<100:
        zeros_original[i]=train_images[k]
        i+=1
    elif train_labels[k]==1 and j<100:
        ones_original[j]=train_images[k]
        j+=1
    k+=1

zeros_32=np.zeros((100,32,32))
ones_32=np.zeros((100,32,32))
for i in range(0,100):
   for j in range(0,28):
       for k in range(0,28):
           zeros_32[i][j+2][k+2]=zeros_original[i][j][k]
           ones_32[i][j+2][k+2]=ones_original[i][j][k]
        
zeros_row_32=np.zeros((100,1024))
ones_row_32=np.zeros((100,1024))
for i in range(100):
    for j in range(32):
        for k in range(32):
            zeros_row_32[i][j*32+k]=zeros_32[i][j][k]
            ones_row_32[i][j*32+k]=ones_32[i][j][k]
            #-128~127のグレースケールへ

from PIL import Image
zeros_8=np.zeros((100,8, 8))
ones_8=np.zeros((100,8,8))
for i in range(100):
    zeros_8[i]=np.asarray(Image.fromarray(np.uint8(zeros_original[i])).resize((8,8)))
    ones_8[i]=np.asarray(Image.fromarray(np.uint8(ones_original[i])).resize((8,8)))

zeros_row_8=np.zeros((100,64))
ones_row_8=np.zeros((100,64))
for i in range(100):
    for j in range(8):
        for k in range(8):
            zeros_row_8[i][j*8+k]=zeros_8[i][j][k]-128
            ones_row_8[i][j*8+k]=ones_8[i][j][k]-128
            #-128~127のグレースケールへ

In [4]:
def function_n2(n,m,ki):#kiは1次元(n**2)ベクトル、kwは2次元(n**2,kw_n)ベクトル
    vertical_of_A=np.zeros(n)
    bit=np.array([[0 for i in range(n)] for j in range(pow(2,n))], dtype=object)
    gate=np.array([[0 for i in range(n)] for j in range(pow(2,n))], dtype=object)
    gate2=np.array([[0 for i in range(n+1)] for j in range(pow(2,n))], dtype=object)#gate2行列を横に1つ拡張してtargetbitの場所を記録する。
    # A=np.array([[0.0 for i in range(pow(2,n))] for j in range(pow(2,n))])#入るのは0,±1/2
    # constant1=np.array([0.0 for i in range(pow(2,n)-2)])#連立方程式の定数部分
    # constant2=np.array([0.0 for i in range(pow(2,n)-2)])#連立方程式の定数部分
    # constant3=np.array([[0.0 for i in range(pow(2,n))] for j in range(pow(2,n))])
    # B=np.array([0.0 for i in range(pow(2,n)-2)])
    # C=np.array([[0.0 for i in range(pow(2,n)-2)] for j in range(pow(2,n)-2)])
    # D=np.array([[0.0 for i in range(pow(2,n))] for j in range(kw_n)])
    # E=np.array([[0.0 for i in range(pow(2,n))] for j in range(kw_n)])
    FLAG=np.array([0 for i in range(pow(2,m))])
    FLAG2=np.array([[1 for i in range(n+m+1)] for j in range(2**m)], dtype=object)
    FLAG3=np.array([[0 for i in range(m)] for j in range(2**m)], dtype=object)
    kw=np.array([pi/4 for i in range(2**m)])
    for i in range(0,pow(2,n)):
        for j in range(0,n):
            bit[i][n-1-j]=bin4(i,n,j)
        #print(bit[i]) #0-pow(2,n)の2進数表記
    #gateを作成する。上から1の数が小さい順に並び変える。今回は下の方のbitを優先してcontrol bitにして適用される画素に偏りをなくす。
    a=1 #aは作成する行列の縦向きの番号,0番目は00...0で入力済み
    for i in range(1,n+1):
        for j in range(0,2**n):
            if sum(bit[2**n-j-1])==i:
                gate[a]=bit[2**n-j-1]
                a+=1
    #gate2を作成する。連立方程式で使う変数は2**n-1個(|00...0>は変化させない)だからその分だけ作る。gateで0になっている部分を1つだけ選んでその場所を一番右に記録する,gateより横に１つ大きい。
    a=0 #0行目は便宜的なもの。いや、Rzゲート使うから必要(20221130追記&コード変更)。
    for i in range(0,2**n):
        for j in range(0,n):
            if gate[i][j]==0:
                for k in range(n):
                    gate2[a][k]=gate[i][k]
                gate2[a][n]=j
                a+=1
                break
                #print(a)
        if a==2**n:
                break
        else:
            continue
    # for i in range(0,2**n):
    #     for j in range(0,2**n):
    #         for k in range(0,n):
    #             vertical_of_A[k]=gate[j][k]-bit[i][k]#これに1が含まれていたらgateがbitを超越しているということ。
    #         if 1 not in vertical_of_A and bit[i][gate2[j][n]]==0:
    #             A[i][j]=-1
    #         elif 1 not in vertical_of_A and bit[i][gate2[j][n]]==1:
    #             A[i][j]=1
    # for i in range(2**n-2):
    #     for j in range(2**n-2):
    #         C[i][j]=A[i+1][j+1]
    # for i in range(2**n-2):
    #     constant1[i]=(pi)*(np.count_nonzero(C[i]!=0))
    # constant2=ki-constant1
    # B=solve(C,constant2)
    #B=np.dot(np.linalg.inv(A),ki-constant)
    #invは行列式が0にならない正則行列の逆行列を求める(行列式が0では逆行列がない)
    #pinvは疑似的に逆行列を求めてくれる。
    #B=np.dot(np.linalg.pinv(A),ki-constant)
    # B=B%(2*pi)

    for i in range(2**m):
        for j in range(m):
            FLAG3[i][j]=bin4(i,m,j)
    for i in range(2**m):
        FLAG[i]=i%m
    for i in range(2**m):
        FLAG2[i][n+FLAG[i]]=0
        FLAG2[i][n+m]=n+FLAG[i]
##ここから
    nqubits = n+m
    state = QuantumState(nqubits)
    state.set_zero_state()
    print(state.get_vector())
    
    # H(1).update_quantum_state(state)
    # q = QuantumRegister(n+m, 'q')
    # c = ClassicalRegister(m, 'c')
    # qc = QuantumCircuit(q, c)

    # for i in range(n+m):
        # qc.h(i)
        # H(i).update_quantum_state(state)
        # qc.barrier(i)
        # Barrier(0).update_quantum_state(state)
    # for i in range(0,pow(2,n)-2):
        # MCRZ(gate2[i+1],n,qc,q,B[i])
    # for i in range(kw_n):
    #     for j in range(1,pow(2,n)):
    #         MCRZ(gate2[j],n,qc,q,E[i][j])
    # qc.append(RZGate(0).control(9),[0,1,2,3,4,5,6,7,8,9])
    state.load(ki)
    for i in range(n):
        H(i).update_quantum_state(state)

    for i in range(2**m):
        for j in range(m):
            if(FLAG3[i][j]==0):
                X(n-m+j).update_quantum_state(state)
                X(n+j).update_quantum_state(state)
        MCRZ(FLAG2[i],n+m,state,kw[i])
        for j in range(m):
            if(FLAG3[i][j]==0):
                X(n-m+j).update_quantum_state(state)
                X(n+j).update_quantum_state(state)

    for i in range(n,n+m):
        H(i).update_quantum_state(state)  
    # for i in range(m):
    #     qc.measure(q[n+i],c[i])


    # MCX(n+1,qc)

    # qc.measure(q[n],c[0])
    # qc.draw("mpl")


    #shots=1048576
    # shots=1024
    # simulator = Aer.get_backend('qasm_simulator')
    # result = execute(qc, backend = simulator, shots=shots, memory=True).result()
    result = state.sampling(nqubits)
    # counts = result.get_counts()
    # counts = result.memory(qc)
    print(num_ones)
    num_ones=np.array([0.0 for i in range(m)])
    for i in range(n,n+m):
        num_ones[i] = result[i]/sum(result[n:n+m])
    # for i in range(2**m):
    #     num_ones[i] = (counts.get(bin6(i,m))-shots/(2**m))/(shots/(2**m))
    return num_ones

In [5]:
nqubits = 10
state = QuantumState(nqubits)
state.set_zero_state()
print(state.get_vector())
a=[0,1,2,3,4,5,6,7,8,9]
mcrz=np.array([[0+0j for i in range(1024)] for j in range(1024)])
for i in range(1024):
    mcrz[i][i]=1
# mcrz[2*m][2*m]=E**(-I*degree/2)
# mcrz[2*m+1][2*m+1]=E**(I*degree)
# gate_array=[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]]
print(len(mcrz))
print(len(a))
# mcrz_gate = DenseMatrix(a,mcrz[0:4][0:4])
mcrz_gate = DenseMatrix(a,mcrz)
mcrz_gate.update_quantum_state(state)
print(state)

[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
1024
10
 *** Quantum State ***
 * Qubit Count : 10
 * Dimension   : 1024
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0

In [6]:
# ki1=np.zeros((100,4,16,16,256))
ki1=np.zeros((100,4*16*16*256))
# for i in range(100):
#     for j in range(4):
#         for k in range(8):
#             for l in range(8):
#                 for o in range(256):
#                     ki1[i][j][j//2+3+k][j%2+3+l][o]=zeros_8[i][k][l]/256
for i in range(100):
    for j in range(4):
        for k in range(8):
            for l in range(8):
                for o in range(256):
                    ki1[i][j*16*16*256+(j//2+3+k)*16*256+(j%2+3+l)*256+o]=zeros_8[i][k][l]/256
# ki1=np.array([1.5 for i in range(62)])
num_ones=np.array([0.0 for i in range(8)])
print(len(np.ravel(ki1[0])))
# original_num_ones=function_n2(10,8,np.ravel(ki1[0]))
original_num_ones=function_n2(10,8,ki1[0])
print(original_num_ones)
# qc.draw("mpl")
# for i in range(256):
#     for j in range(8):
#         if(bin6(i,10)=='1'):
#             num_ones[j]=num_ones[j]+original_num_ones[i]
num_ones=original_num_ones
print(num_ones)
ki2=np.array([pi/2 for i in range(64)])
ki2[2]=ki2[2]+num_ones[0] ## # 0 #
ki2[5]=ki2[5]+num_ones[1] ## 1 2 3
ki2[6]=ki2[6]+num_ones[2] #4 5 6 #
ki2[7]=ki2[7]+num_ones[3] ## 7 # #
ki2[8]=ki2[8]+num_ones[4]
ki2[9]=ki2[9]+num_ones[5]
ki2[10]=ki2[10]+num_ones[6] 
ki2[13]=ki2[13]+num_ones[7]
ki2[29]=ki2[29]+num_ones[0] ## # 1 #
ki2[18]=ki2[18]+num_ones[1] ## 2 3 4
ki2[21]=ki2[21]+num_ones[2] #5 6 7 #
ki2[22]=ki2[22]+num_ones[3] ## 0 # #
ki2[23]=ki2[23]+num_ones[4]
ki2[24]=ki2[24]+num_ones[5]
ki2[25]=ki2[25]+num_ones[6]
ki2[26]=ki2[26]+num_ones[7]
ki2[42]=ki2[42]+num_ones[0] ## # 2 #
ki2[45]=ki2[45]+num_ones[1] ## 3 4 5
ki2[34]=ki2[34]+num_ones[2] #6 7 0 #
ki2[37]=ki2[37]+num_ones[3] ## 1 # /
ki2[38]=ki2[38]+num_ones[4]
ki2[39]=ki2[39]+num_ones[5]
ki2[40]=ki2[40]+num_ones[6]
ki2[41]=ki2[41]+num_ones[7]
ki2[57]=ki2[57]+num_ones[0] ## # 3 #
ki2[58]=ki2[58]+num_ones[1] ## 4 5 6
ki2[61]=ki2[61]+num_ones[2] #7 0 1 #
ki2[50]=ki2[50]+num_ones[3] ## 2 # #
ki2[53]=ki2[53]+num_ones[4]
ki2[54]=ki2[54]+num_ones[5]
ki2[55]=ki2[55]+num_ones[6]
ki2[56]=ki2[56]+num_ones[7]
for i in range(64):
    for j in range(4):
        ki2[(63-i)*4+j]=ki2[63-i]
original_num_ones=function_n2(6,4,ki2)
print(original_num_ones)
# qc.draw("mpl")
num_ones=np.array([0.0 for i in range(4)])
# for i in range(16):
#     for j in range(4):
#         if(bin6(i,4)=='1'):
#             num_ones[j]=num_ones[j]+original_num_ones[i]
num_ones=original_num_ones
print(num_ones)
# sum=num_ones[0]+num_ones[1]+num_ones[2]+num_ones[3]
print(sum(num_ones))

262144
[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
36
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 10]


ValueError: matrix dims is not consistent.

In [7]:
def multi(n):
    p = Pool(32) #max process number :10
    result = p.map(main_calculation, range(n))
    # result = map(main_calculation, range(n))
    #either commentout upper 2lines or latter one.
    return n,result#interator, take out by for-sentences.
def main():
    i,data = multi(314)
    for j in data:
        pass
results_zeros_0=np.array([0.0 for i in range (314)])
bottom=np.array([0.0 for i in range (314)])

def main_calculation(i):
    kw_32=np.array([i/100 for j in range (pow(2,10))])
    results_zeros_0[i]=function_n(10,zeros_row_32[0]*pi/256,kw_32,1)[0]
    bottom[i]=i/100
    print(f"i={i}:{results_zeros_0[i]}")
    return results_zeros_0[i]
    
# main()
# plt.plot(bottom,results_zeros_0)